In [1]:
import pandas as pd
import numpy as np
from src import util
import warnings
warnings.filterwarnings(action='once')

In [2]:
code = 12025  # código do curso de ciência da computação da ufpa

In [3]:
csv = pd.read_csv("data/enade/3.DADOS/MICRODADOS_ENADE_2017.txt", 
                  sep=";", decimal =',', 
                  dtype={"DS_VT_ESC_OFG":str, 
                         "DS_VT_ESC_OCE": str,
                         "DS_VT_ACE_OCE": str})
classificacao_tema = pd.read_csv("data/classificacao_charao.csv")

In [4]:
csv.head()

,NU_ANO,CO_IES,CO_CATEGAD,CO_ORGACAD,CO_GRUPO,CO_CURSO,CO_MODALIDADE,CO_MUNIC_CURSO,CO_UF_CURSO,CO_REGIAO_CURSO,...,QE_I72,QE_I73,QE_I74,QE_I75,QE_I76,QE_I77,QE_I78,QE_I79,QE_I80,QE_I81
0,2017,1,1,10028,5710,3,1,5103403,51,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017,1,1,10028,5710,3,1,5103403,51,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017,1,1,10028,5710,3,1,5103403,51,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017,1,1,10028,5710,3,1,5103403,51,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017,1,1,10028,5710,3,1,5103403,51,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Filtrar dados apenas dos estudantes do curso de ciência da computação da ufpa
csv_curso = csv.loc[csv["CO_CURSO"] == 12025]


In [6]:
# Mapear codigo de presença pra label
new_mapping = csv_curso["TP_PRES"].map({222:"Ausente", 555:"Presente"}).values
csv_curso.loc[:, "TP_PRES_LABEL"] = new_mapping



/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


In [7]:
csv_curso["TP_PRES_LABEL"].value_counts()

Presente    34
Ausente     14
Name: TP_PRES_LABEL, dtype: int64

In [8]:
# Filtrar dados apenas dos estudantes que prestaram o exame
csv_curso_presentes = csv_curso.loc[csv_curso["TP_PRES_LABEL"] == "Presente"]

In [9]:
# Cria uma coluna pra cada questão, indicando se o aluno acertou ou não a mesma. O valor fica NaN caso a questão
# tenha sido anulada
csv_curso_presentes_2 = util.add_columns_objective_score(csv_curso_presentes)


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


In [10]:
# Filtrar dados das questões apenas do exame de 2017
classificacao_tema = classificacao_tema.loc[classificacao_tema["ano"] == 2017]


In [11]:
subjects = util.get_subjects(classificacao_tema)
subjects

array(['Algoritmos e Estruturas de Dados',
       'Arquitetura de Computadores e Sistemas Operacionais',
       'Banco de Dados', 'Compiladores',
       'Computação Gráfica e Processamento de Imagem',
       'Engenharia de Software e Interação Homem-Computador',
       'Fundamentos e Técnicas de Programação',
       'Inteligência Artificial e Computacional',
       'Lógica e Matemática Discreta',
       'Paradigmas de Linguagens de Programação', 'Redes de Computadores',
       'Sistemas Digitais', 'Sistemas Distribuídos',
       'Teoria da Computação', 'Teoria dos Grafos',
       'Ética, Computador e Sociedade'], dtype=object)

In [12]:
# Adiciona novas colunas pra cada tema do exame (Algoritmos, Grafos, etc) e atribui a nota do aluno no tema 
# respectivo
csv_curso_presentes = util.add_all_score_subjects(csv_curso_presentes, 
                                                  classificacao_tema)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


In [13]:
# medias dos alunos por cada tema, junto com o número de questões
num_questions = []
for subject in subjects:
    num_subject_questions = len(util.get_subject_valid_questions(subject, 
                                                                 classificacao_tema,
                                                                 csv_curso_presentes))
    num_questions.append(num_subject_questions)
    
    
subjects_labels = [f"SCORE_{x}" for x in subjects]

mean_by_subject = csv_curso_presentes[subjects_labels].mean().values

data = np.array([mean_by_subject, num_questions]).T



subject_df = pd.DataFrame(data=data, index=subjects, 
                          columns=["Nota %", "Nº Questões"])
subject_df["Nº Questões"] = subject_df["Nº Questões"].astype(int)

subject_df.sort_values(by=["Nota %"])


,Nota %,Nº Questões
Teoria da Computação,13.235294,2
Banco de Dados,30.882353,2
Compiladores,32.352941,1
Algoritmos e Estruturas de Dados,35.000000,6
Arquitetura de Computadores e Sistemas Operacionais,35.294118,3
Computação Gráfica e Processamento de Imagem,41.176471,1
Sistemas Distribuídos,41.176471,2
Teoria dos Grafos,43.676471,2
Fundamentos e Técnicas de Programação,47.058824,1
Lógica e Matemática Discreta,47.058824,1


In [14]:
# porcentagem de questões objetivas marcadas em branco e rasuradas
all_questions_to_do = "".join(csv_curso_presentes["DS_VT_ESC_OFG"]) + "".join(csv_curso_presentes["DS_VT_ESC_OCE"])
num_blank = all_questions_to_do.count('.')
num_deletion = all_questions_to_do.count('*')
print(f"{round(num_blank*100/len(all_questions_to_do), 2)}% questões marcadas em branco" 
      f" (apenas um aluno deixou UMA questão em branco)")
print(f"{num_deletion*100/len(all_questions_to_do)}% rasuradas ")

0.08% questões marcadas em branco (apenas um aluno deixou UMA questão em branco)
0.0% rasuradas 


In [15]:
# Porcentagem de alunos que não realizou nenhuma questão discursiva
general_formation_blank = csv_curso_presentes["TP_PR_DI_FG"] == 333
specific_formation_blank = csv_curso_presentes["TP_PR_DI_CE"] == 333
discursive_blank = general_formation_blank & specific_formation_blank
percentage_dis_blank = discursive_blank.mean()*100
percentage_dis_blank = round(percentage_dis_blank, 2)
print(f"{percentage_dis_blank}% dos alunos deixaram toda a prova discursiva (5 questões) em branco")
print("O que dá 2 alunos")

5.88% dos alunos deixaram toda a prova discursiva (5 questões) em branco
O que dá 2 alunos


In [16]:
discursive_general_labels = [f"TP_SFG_D{x}" for x in range(1, 3)]
discursive_specific_labels = [f"TP_SCE_D{x}" for x in range(1, 4)]
labels = discursive_general_labels + discursive_specific_labels
blank_discursive = csv_curso_presentes[labels] == 333
num_blank_discursive = sum(blank_discursive.sum())
percentage_blank_discursive = num_blank_discursive / np.prod(blank_discursive.shape)
percentage_blank_discursive *= 100
percentage_blank_discursive = round(percentage_blank_discursive, 2)
print(f"{percentage_blank_discursive}% das questões discursivas foram deixadas em branco")


25.29% das questões discursivas foram deixadas em branco
